# Sudoku Solver - Using the Crosshatching Method

In [322]:
import numpy as np

In [323]:
from tkinter import *
from tkinter import ttk

### Tk: Create puzzle GUI

In [324]:
root = Tk()
root.title("Sudoku Solver")

mainframe = ttk.Frame(root, padding="15")
mainframe.grid(column=0, row=0, stick=(N, W, E, S))

mainframe.columnconfigure(0, weight=1)
mainframe.rowconfigure(0, weight=1)

In [325]:
ttk.Label(mainframe, text="Sudoku Solver", anchor="center").grid(column=1, columnspan=10, row=0, sticky=(W,E))

### Tk: Create a grid of Entry widgets and declare each Entry as a Tk variable - StringVar()

In [326]:
position ={}
pos = 1
for x in range(0,9):
    for y in range(0,9):
        position[pos] = StringVar()
        ttk.Entry(mainframe, width=2, textvariable=position[pos]).grid(row=x+1, column=y+1, sticky=(W,E))
        pos += 1

### Numpy: Create board and set sub-grids to be view of the main board

In [327]:
board = np.zeros((9,9), dtype=np.int64)
board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [328]:
a = board[0:3, 0:3]
b = board[0:3, 3:6]
c = board[0:3, 6:9]
d = board[3:6, 0:3]
e = board[3:6, 3:6]
f = board[3:6, 6:9]
g = board[6:9, 0:3]
h = board[6:9, 3:6]
i = board[6:9, 6:9]

In [329]:
subGrids = [a,b,c,d,e,f,g,h,i]

### Tk: Assign the Entry value to the board elements
- If Entry is empty (len(position[i].get()) == 0) the assign that element to zero 0
- If filled assign board element to that value

In [330]:
def set_puzzle(*args):    
    idx = 1
    for x in range(0,9):
        for y in range(0,9):
            if len(position[idx].get()) == 0:
                board[x,y] = 0
            else:
                value = int(position[idx].get())
                board[x,y] = value
            idx += 1

### Numpy: Function to find all possible numbers for each sub-grid
- Using numpy.setdiff1d(array1, array2) - Return the sorted, unique values in array1 that are not in array2 (Order matters)
- Convert array of possible choices into a single number

In [331]:
def possibleEntry(arr2, arr1=np.arange(1,10)):
    if(len(arr1) == 0): #if comparing array is empty, just return the other array
        return arr2
    else:
        choice = np.setdiff1d(arr1, arr2)
        if(len(choice) > 0): #if not empty join into a single number
            choice = int(''.join(str(i) for i in choice))
            return choice
        else: #if empty, return empty array
            return choice

### Numpy: Eliminate the numbers in the row and column and assign possible numbers to "empty" elements

In [332]:
def crosshatch(board):
    #set array to add possible numbers to, based on row and column
    pool = np.empty(0, dtype=np.int64)
    
    board_length = board.shape[0]
    for x in range(board_length): #row
        for y in range(board_length): #column
            #if element is longer than 1 digit check its row and column
            if (len(str(board[x,y])) > 1 ): 

                pool = np.append(pool, board[x, :]) #append all the numbers in that element's row
                pool = np.append(pool, board[:, y]) #append all the numbers in that element's column
                pool = pool[ pool < 10] #eliminate any number laster than 9

                current = [int(i) for i in str(board[x,y])] #convert current element into an array of numbers
                np.asarray(current, dtype=np.int64)
            
                #assign the element to a new possibleEntry
                board[x,y] = possibleEntry(pool, current)
            
                #reset the pool for the next element
                pool = np.empty(0, dtype=np.int64)
    
    return board
            
        

### Numpy: Function to check if puzzle is complete
- Returns False if grid contains zeros or numbers larger than 9

In [333]:
def checkGrid(board):
    board_length = board.shape[0]
    for x in range(board_length): #row
        for y in range(board_length): #column
            if (board[x,y] > 9 or int(board[x,y]) == 0):
                return False
    return True
    

### Tk: Solve function - once solved assign numbers of the board elements to each Entry widget to dsiplay on GUI

In [334]:
def solve(*args): 
    set_puzzle()
    while checkGrid(board) == False:
        for i in subGrids:
            i[i == 0] = possibleEntry(i)
            crosshatch(board)
        
    idx = 1
    for x in range(0,9):
        for y in range(0,9):
            position[idx].set(board[x,y])
            idx += 1

In [335]:
board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

### Tk: Add bottuns on the bottom and assign the functions

In [336]:
ttk.Button(mainframe, text="Solve", command=solve).grid(column=1, columnspan=5, row=12, sticky=(W, E))
ttk.Button(mainframe, text="Clear").grid(column=6, columnspan=5, row=12, sticky=(W, E))

### Tk: End GUI with Tk mainloop

In [337]:
root.mainloop()